# Clasificar proyectos ANID en salud u otro

El objetivo de este notebook es clasificar cada proyecto ANID en `salud` si es que aborda algún tema (cualquiera) relacionado a salud, u `otro`, en caso contrario.

In [ ]:
!pip install openai
!pip install langchain
from IPython.display import clear_output
clear_output()

In [ ]:
import os
import numpy as np
import pandas as pd
from csv import writer
from tqdm import tqdm
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from google.colab import userdata

## Cargar datos

In [ ]:
data_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/ANID/Datos"
save_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/ANID/Resultados"
df = pd.read_excel(os.path.join(data_dir, "BDH_HISTORICA.xlsx"))
df

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,GRUPO_DE_EVALUACION,DURACION_MESES,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO
0,ECOS190041,35152,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,MOVILIDAD - ECOS,PROGRAMA DE COOPERACION CIENTIFICA ECOS,2019,2020,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,CIENCIAS SOCIALES,...,NO APLICA,36.0,PERSONA NATURAL,JIMENA TORRES,MUJER,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,9280,NaN
1,FP200006,35312,"REDES, ESTRATEGIA Y CONOCIMIENTO",INFORMACION CIENTIFICA,FPRC,FONDO DE PUBLICACION DE REVISTAS CIENTIFICAS 2020,2020,2020,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,SIN INFORMACION,...,NO APLICA,24.0,PERSONA JURIDICA,FABIANA MARTIN,MUJER,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,12005.582,NaN
2,FOVI210059,37149,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,CIENCIAS NATURALES,...,NO APLICA,12.0,PERSONA JURIDICA,LAURA SANCHEZ-JARDON,MUJER,FUNDACION OMORA ONG,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,20000,NaN
3,FOVI210061,37151,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,CIENCIAS NATURALES,...,NO APLICA,12.0,PERSONA JURIDICA,SERGIO RADIC,HOMBRE,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,20000,NaN
4,ED230064,34491,NO APLICA,EXPLORA,PROYECTOS DE DIVULGACION Y VALORACION DE LA CI...,XXIII CONCURSO NACIONAL DE PROYECTOS EXPLORA P...,2019,2019,PODCAST ANTARTICA,MULTIDISCIPLINARIO,...,NO APLICA,12.0,PERSONA NATURAL,PAULINA ROJAS,MUJER,INSTITUTO ANTARTICO CHILENO (INACH),AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,22000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42073,CL-2023B-012,42074,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,RAPID RESPONSE TOO SPECTROSCOPIC OBSERVATIONS ...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"PIGNATA, GIULIANO",HOMBRE,UNIVERSIDAD NACIONAL ANDRÉS BELLO,RM,13. METROPOLITANA,0,NO ENTREGA RECURSOS
42074,CL-2023B-015,42075,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING CLOUDS AND HAZES IN AN INFLATED WARM-J...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"RAJKUMAR, ANITHA RAJ",HOMBRE,UNIVERSIDAD DE ATACAMA,NORTE,03. ATACAMA,0,NO ENTREGA RECURSOS
42075,CL-2023B-016,42076,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING THE FORMATION HISTORIES OF NSCS IN THE...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"NAYAK, PRASANTA KUMAR",MUJER,PONTIFICIA UNIVERSIDAD CATÓLICA DE CHILE,RM,13. METROPOLITANA,0,NO ENTREGA RECURSOS
42076,CL-2023B-017,42077,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,"ELUCIDATING THE NATURE OF GRB\,220831A'S OPTIC...",CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"TEJOS, NICOLAS",HOMBRE,PONTIFICIA UNIVERSIDAD CATÓLICA DE VALPARAÍSO,CENTRO,05. VALPARAISO,0,NO ENTREGA RECURSOS


## Preprocesar datos

In [ ]:
df.columns

Index(['CODIGO_PROYECTO', 'N', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'AGNO_CONCURSO', 'AGNO_FALLO', 'NOMBRE_PROYECTO',
       'AREA_OCDE', 'DISCIPLINA_DETALLE', 'GRUPO_DE_EVALUACION',
       'DURACION_MESES', 'TIPO_BENEFICIARIO', 'NOMBRE_RESPONSABLE', 'SEXO',
       'INSTITUCION_PRINCIPAL', 'MACROZONA_MINCIENCIA', 'REGION_EJECUCION',
       'MONTO_ADJUDICADO', 'NOTA_MONTO'],
      dtype='object')

In [ ]:
df[df["NOMBRE_PROYECTO"].isna()]

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,GRUPO_DE_EVALUACION,DURACION_MESES,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO
10595,BDE041,10564,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADO EN EL EXTRANJERO,CONCURSO DE BECAS DE DOCTORADO EN EL EXTRANJER...,2004,2004,NaN,MULTIDISCIPLINARIO,...,NO APLICA,NaN,PERSONA NATURAL,ANGELA MARCELA GANZ BUSTOS,MUJER,INRIA SOPHIA-ANTIPOLIS,EXTRANJERO,EXTRANJERO,NaN,NaN
10596,BDE0410,10565,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADO EN EL EXTRANJERO,CONCURSO DE BECAS DE DOCTORADO EN EL EXTRANJER...,2004,2004,NaN,CIENCIAS NATURALES,...,NO APLICA,NaN,PERSONA NATURAL,CLAUDIA ANDREA MARQUEZ THOMAS,MUJER,UNIVERSIDAD PARIS 7 DENIS DIDEROT,EXTRANJERO,EXTRANJERO,NaN,NaN
10597,BDE0411,10566,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADO EN EL EXTRANJERO,CONCURSO DE BECAS DE DOCTORADO EN EL EXTRANJER...,2004,2004,NaN,CIENCIAS AGRICOLAS,...,NO APLICA,NaN,PERSONA NATURAL,MARITZA REGINA REYES CARREÑO,MUJER,INSTITUTO NACIONAL DE INVESTIGACION AGRONOMICA...,SIN INFORMACION,SIN INFORMACION,NaN,NaN
10598,BDE0412,10567,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADO EN EL EXTRANJERO,CONCURSO DE BECAS DE DOCTORADO EN EL EXTRANJER...,2004,2004,NaN,INGENIERIA Y TECNOLOGIA,...,NO APLICA,NaN,PERSONA NATURAL,LUIS CRISTOBAL ROJAS GONZALEZ,HOMBRE,UNIVERSIDAD PIERRE ET MARIE CURIE,EXTRANJERO,EXTRANJERO,NaN,NaN
10599,BDE0413,10568,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADO EN EL EXTRANJERO,CONCURSO DE BECAS DE DOCTORADO EN EL EXTRANJER...,2004,2004,NaN,INGENIERIA Y TECNOLOGIA,...,NO APLICA,NaN,PERSONA NATURAL,ESTEBAN PATRICIO SAEZ ROBERT,HOMBRE,ESCUELA CENTRAL PARIS,EXTRANJERO,EXTRANJERO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12977,BDN0695,12946,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADOS NACIONALES,CONCURSO DE BECAS DE DOCTORADOS NACIONALES AÑO...,2006,2006,NaN,CIENCIAS AGRICOLAS,...,NO APLICA,NaN,PERSONA NATURAL,CLAUDIA ISABELHARCHA CORTES,MUJER,UNIVERSIDAD AUSTRAL DE CHILE,SUR,14. LOS RIOS,NaN,NaN
12978,BDN0696,12947,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADOS NACIONALES,CONCURSO DE BECAS DE DOCTORADOS NACIONALES AÑO...,2006,2006,NaN,CIENCIAS NATURALES,...,NO APLICA,NaN,PERSONA NATURAL,DENISE ALICIA HAUSSMANN BIELEFELD,MUJER,UNIVERSIDAD AUSTRAL DE CHILE,SUR,14. LOS RIOS,NaN,NaN
12979,BDN0697,12948,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADOS NACIONALES,CONCURSO DE BECAS DE DOCTORADOS NACIONALES AÑO...,2006,2006,NaN,CIENCIAS NATURALES,...,NO APLICA,NaN,PERSONA NATURAL,RICARDO ANDRES HENRIQUEZ CORREA,HOMBRE,UNIVERSIDAD DE CHILE,RM,13. METROPOLITANA,NaN,NaN
12980,BDN0698,12949,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADOS NACIONALES,CONCURSO DE BECAS DE DOCTORADOS NACIONALES AÑO...,2006,2006,NaN,INGENIERIA Y TECNOLOGIA,...,NO APLICA,NaN,PERSONA NATURAL,LUIS ANDRES HENRIQUEZ VARGAS,HOMBRE,UNIVERSIDAD DE SANTIAGO DE CHILE,RM,13. METROPOLITANA,NaN,NaN


Tenemos 574 filas sin nombre de proyecto; parecen ser de adjudicaciones de becas. Además, a diferencia de CORFO, no contamos con la descripción de los proyectos, sino que sólo tenemos los títulos.

In [ ]:
df_valid = df[~df["NOMBRE_PROYECTO"].isna()]
len(df_valid)

41504

In [ ]:
df_valid = df_valid[df_valid['AGNO_FALLO'] >= 2012]
df_valid

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,GRUPO_DE_EVALUACION,DURACION_MESES,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO
0,ECOS190041,35152,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,MOVILIDAD - ECOS,PROGRAMA DE COOPERACION CIENTIFICA ECOS,2019,2020,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,CIENCIAS SOCIALES,...,NO APLICA,36.0,PERSONA NATURAL,JIMENA TORRES,MUJER,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,9280,NaN
1,FP200006,35312,"REDES, ESTRATEGIA Y CONOCIMIENTO",INFORMACION CIENTIFICA,FPRC,FONDO DE PUBLICACION DE REVISTAS CIENTIFICAS 2020,2020,2020,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,SIN INFORMACION,...,NO APLICA,24.0,PERSONA JURIDICA,FABIANA MARTIN,MUJER,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,12005.582,NaN
2,FOVI210059,37149,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,CIENCIAS NATURALES,...,NO APLICA,12.0,PERSONA JURIDICA,LAURA SANCHEZ-JARDON,MUJER,FUNDACION OMORA ONG,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,20000,NaN
3,FOVI210061,37151,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,CIENCIAS NATURALES,...,NO APLICA,12.0,PERSONA JURIDICA,SERGIO RADIC,HOMBRE,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,20000,NaN
4,ED230064,34491,NO APLICA,EXPLORA,PROYECTOS DE DIVULGACION Y VALORACION DE LA CI...,XXIII CONCURSO NACIONAL DE PROYECTOS EXPLORA P...,2019,2019,PODCAST ANTARTICA,MULTIDISCIPLINARIO,...,NO APLICA,12.0,PERSONA NATURAL,PAULINA ROJAS,MUJER,INSTITUTO ANTARTICO CHILENO (INACH),AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,22000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42073,CL-2023B-012,42074,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,RAPID RESPONSE TOO SPECTROSCOPIC OBSERVATIONS ...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"PIGNATA, GIULIANO",HOMBRE,UNIVERSIDAD NACIONAL ANDRÉS BELLO,RM,13. METROPOLITANA,0,NO ENTREGA RECURSOS
42074,CL-2023B-015,42075,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING CLOUDS AND HAZES IN AN INFLATED WARM-J...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"RAJKUMAR, ANITHA RAJ",HOMBRE,UNIVERSIDAD DE ATACAMA,NORTE,03. ATACAMA,0,NO ENTREGA RECURSOS
42075,CL-2023B-016,42076,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING THE FORMATION HISTORIES OF NSCS IN THE...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"NAYAK, PRASANTA KUMAR",MUJER,PONTIFICIA UNIVERSIDAD CATÓLICA DE CHILE,RM,13. METROPOLITANA,0,NO ENTREGA RECURSOS
42076,CL-2023B-017,42077,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,"ELUCIDATING THE NATURE OF GRB\,220831A'S OPTIC...",CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"TEJOS, NICOLAS",HOMBRE,PONTIFICIA UNIVERSIDAD CATÓLICA DE VALPARAÍSO,CENTRO,05. VALPARAISO,0,NO ENTREGA RECURSOS


Dejaremos sólo los proyectos de 2012 en adelante.

## Clasificación

Inicializar modelo

In [ ]:
api_key = userdata.get('openai_api_key')
chat = ChatOpenAI(model_name="gpt-4",
                    openai_api_key=api_key,
                    temperature=0,
                    max_tokens=3500)
chat

Le daremos al LLM una lista de instrucciones, pidiéndole clasificar cada proyecto en `cáncer`, `diabetes`, `cardiaca` u `otro`. Especificaremos que los proyectos pueden estar en inglés o español. Luego, le pasaremos una lista de proyectos para que clasifique. Iteraremos en bloques de 8 proyectos, e iremos guardando de inmediato los resultados.

In [ ]:
system_prompt = ("Eres un experto en evaluación de proyectos de investigación. " +
                 "Recibiras una lista de proyectos, cada uno empezando con un guión (-). " +
                 "Los proyectos sólo tienen título, y estos pueden estar en inglés o español. " +
                 "Tu trabajo es determinar, para cada proyecto, si habla de salud u otro tema. " +
                 "Si el tema del proyecto es salud humana, debes responder 'salud'. " +
                 "Si el tema del proyecto no es salud humana, debes responder 'otro'. " +
                 "No pongas guiones al principio de cada respuesta. Sólo debes responder la clasificación correcta en cada línea.")

system_message_prompt = SystemMessage(content=system_prompt)
n_projects_prompt = 8

filepath = os.path.join(save_dir, "clasificaciones_anid_salud_otro.csv")
if not os.path.exists(filepath):
  with open(filepath, "w") as fp:
    print("Creando archivo...")
    csv_writer = writer(fp)
    header = ["código", "glosa", "clasificación"]
    csv_writer.writerow(header)


for iii in tqdm(range(0, len(df_valid), n_projects_prompt)):
  current_df = df_valid.iloc[iii: iii + n_projects_prompt]
  projects = "\n".join(("- " + current_df["NOMBRE_PROYECTO"]).values)
  messages = [
    system_message_prompt, HumanMessage(content=projects)
  ]
  ai_response = chat(messages)
  responses = ai_response.content.split("\n")
  if len(current_df) != len(responses):
    print(f"Distinto número de respuestas que de proyectos")
    print(iii, iii+1)
  else:
    rows = list(zip(current_df["CODIGO_PROYECTO"].values, current_df["NOMBRE_PROYECTO"], responses))
    with open(filepath, "a") as fp:
      csv_writer = writer(fp)
      csv_writer.writerows(rows)

Creando archivo...


  0%|          | 0/2634 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
 33%|███▎      | 862/2634 [45:40<59:40,  2.02s/it]  

Distinto número de respuestas que de proyectos
6888 6889


 76%|███████▌  | 2006/2634 [1:49:33<3:50:17, 22.00s/it]

Distinto número de respuestas que de proyectos
16040 16041


 77%|███████▋  | 2020/2634 [1:50:20<30:58,  3.03s/it]

Distinto número de respuestas que de proyectos
16152 16153


100%|██████████| 2634/2634 [2:21:19<00:00,  3.22s/it]


### Datos faltantes

Vemos que hubo algunas iteraciones que fallaron, así que intentaremos de nuevo procesar esos datos.

In [ ]:
df_missing = df_valid.iloc[np.r_[6888:6896, 16040:16048, 16152:16160]]
df_missing

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,GRUPO_DE_EVALUACION,DURACION_MESES,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO
27900,REDES150010,27869,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,REDES INTERNACIONALES - PROGRAMA DE APOYO A LA...,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,2015,2016,INTERNATIONAL NETWORK ON TERRITORIAL JUSTICE.,CIENCIAS SOCIALES,...,NO APLICA,18.0,PERSONA JURIDICA,OSCAR MAC-CLURE,HOMBRE,UNIVERSIDAD DE LOS LAGOS,SUR,10. LOS LAGOS,16000,NaN
27901,REDES150011,27870,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,REDES INTERNACIONALES - PROGRAMA DE APOYO A LA...,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,2015,2016,BIOREMEDIATION OF NATURAL ENVIRONMENTS THROUGH...,CIENCIAS NATURALES,...,NO APLICA,18.0,PERSONA JURIDICA,HECTOR RAMIREZ CABRERA,HOMBRE,UNIVERSIDAD DE CHILE,RM,13. METROPOLITANA,16000,NaN
27902,REDES150014,27871,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,REDES INTERNACIONALES - PROGRAMA DE APOYO A LA...,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,2015,2016,ALGAL BIOREFINERY NETWORK: FOODS OF NORWAY-ICD...,INGENIERIA Y TECNOLOGIA,...,NO APLICA,18.0,PERSONA JURIDICA,MARIA LIENQUEO CONTRERAS,MUJER,UNIVERSIDAD DE CHILE,RM,13. METROPOLITANA,16000,NaN
27903,REDES150021,27872,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,REDES INTERNACIONALES - PROGRAMA DE APOYO A LA...,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,2015,2016,"TRUTH-TELLING: VIOLENCE, MEMORY AND HUMAN RIGH...",HUMANIDADES,...,NO APLICA,18.0,PERSONA JURIDICA,LUCERO DE VIVANCO ROCA REY,MUJER,UNIVERSIDAD ALBERTO HURTADO,RM,13. METROPOLITANA,16000,NaN
27904,REDES150024,27873,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,REDES INTERNACIONALES - PROGRAMA DE APOYO A LA...,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,2015,2016,PALEORECORDS OF DUST IN SOUTH AMERICA,CIENCIAS NATURALES,...,NO APLICA,18.0,PERSONA JURIDICA,FABRICE LAMBERT,HOMBRE,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,RM,13. METROPOLITANA,15800,NaN
27905,REDES150027,27874,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,REDES INTERNACIONALES - PROGRAMA DE APOYO A LA...,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,2015,2016,RED DE INVESTIGACION DE LA BIODIVERSIDAD ANTAR...,CIENCIAS NATURALES,...,NO APLICA,18.0,PERSONA JURIDICA,ELIE POULIN,HOMBRE,UNIVERSIDAD DE CHILE,RM,13. METROPOLITANA,16000,NaN
27906,REDES150030,27875,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,REDES INTERNACIONALES - PROGRAMA DE APOYO A LA...,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,2015,2016,POSTHARVEST SYSTEMS BIOLOGY: HANDS ON INTEGRAT...,CIENCIAS AGRICOLAS,...,NO APLICA,18.0,PERSONA JURIDICA,ROMINA PEDRESCHI PLASENCIA,MUJER,PONTIFICIA UNIVERSIDAD AGRARIA LA MOLINA,CENTRO,05. VALPARAISO,15500,NaN
27907,REDES150034,27876,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,REDES INTERNACIONALES - PROGRAMA DE APOYO A LA...,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,2015,2016,ADVANCED VECTOR CONTROL TECHNIQUES FOR HIGH AC...,INGENIERIA Y TECNOLOGIA,...,NO APLICA,18.0,PERSONA JURIDICA,SYED ALI KAMRAN SHAH JAFRI,HOMBRE,UNIVERSIDAD DE TALCA,CENTRO SUR,07. MAULE,16000,NaN
37052,SUC210083,37036,INVESTIGACION APLICADA,CORFO,START UP CIENCIA,STARTUP CIENCIA 2021,2021,2021,TEST INMUNODIAGNOSTICO DE FLUJO LATERAL PARA L...,CIENCIAS AGRICOLAS,...,NO APLICA,12.0,PERSONA JURIDICA,VIANA MANRIQUE,MUJER,EMBIOTEC SPA,CENTRO SUR,08. BIOBIO,119986,NaN
37053,SUC210087,37037,INVESTIGACION APLICADA,CORFO,START UP CIENCIA,STARTUP CIENCIA 2021,2021,2021,PLATAFORMA TECNOLOGICA INTEGRADA DE MEJORAMIEN...,CIENCIAS AGRICOLAS,...,NO APLICA,12.0,PERSONA JURIDICA,FRANCISCA CASTILLO,MUJER,NEOCROP TECHNOLOGIES SPA.,SUR,14. LOS RIOS,120000,NaN


Una hipótesis inicial es que el proceso falló en esas iteraciones porque el prompt era muy largo. Así que, en vez de grupos de 8 proyectos, iteraremos en grupos de 2.

In [ ]:
n_projects_prompt = 2
filepath = os.path.join(data_dir, "clasificaciones_anid_salud.csv")
system_prompt = ("Eres un experto en evaluación de proyectos de investigación. " +
                 "Recibiras una lista de proyectos, cada uno empezando con un guión (-). " +
                 "Los proyectos sólo tienen título, y estos pueden estar en inglés o español. " +
                 "Tu trabajo es determinar para cada proyecto si habla de cáncer, de diabetes, de enfermedades cardiacas u otro tema. " +
                 "Si el tema del proyecto es cáncer, debes responder 'cáncer'. " +
                 "Si el tema del proyecto es diabetes, debes responder 'diabetes'. " +
                 "Si el tema del proyecto es enfermedades cardiacas, debes responder 'cardiaca'. " +
                 "Si el tema del proyecto es algo distinto, debes responder 'otro.'. " +
                 "No pongas guiones al principio de cada respuesta. Sólo debes responder la clasificación correcta en cada línea.")

system_message_prompt = SystemMessage(content=system_prompt)


for iii in tqdm(range(0, len(df_missing), n_projects_prompt)):
  current_df = df_missing.iloc[iii: iii + n_projects_prompt]
  projects = "\n".join(("- " + current_df["NOMBRE_PROYECTO"]).values)
  messages = [
    system_message_prompt, HumanMessage(content=projects)
  ]
  ai_response = chat(messages)
  responses = ai_response.content.split("\n")
  if len(current_df) != len(responses):
    print(f"Distinto número de respuestas que de proyectos")
    print(iii, iii + 10)
  else:
    rows = list(zip(current_df["CODIGO_PROYECTO"].values, current_df["NOMBRE_PROYECTO"], responses))
    with open(filepath, "a") as fp:
      csv_writer = writer(fp)
      csv_writer.writerows(rows)

  0%|          | 0/12 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
 58%|█████▊    | 7/12 [00:10<00:06,  1.35s/it]

Distinto número de respuestas que de proyectos
12 22


 92%|█████████▏| 11/12 [00:15<00:01,  1.40s/it]

Distinto número de respuestas que de proyectos
20 30


100%|██████████| 12/12 [00:16<00:00,  1.40s/it]


Dos iteraciones fallaron

In [ ]:
df_missing2 = df_missing.iloc[np.r_[12:14, 20:22]]
df_missing2

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,GRUPO_DE_EVALUACION,DURACION_MESES,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO
37056,SUC210102,37040,INVESTIGACION APLICADA,CORFO,START UP CIENCIA,STARTUP CIENCIA 2021,2021,2021,PNX3: UNA REVOLUCION EN EL TRATAMIENTO DEL DOL...,CIENCIAS MEDICAS Y DE LA SALUD,...,NO APLICA,12.0,PERSONA JURIDICA,DAVID BRAVO,HOMBRE,PANNEX THERAPEUTICS SPA,RM,13. METROPOLITANA,106960,NaN
37057,SUC210108,37041,INVESTIGACION APLICADA,CORFO,START UP CIENCIA,STARTUP CIENCIA 2021,2021,2021,VALIDACION DE UN ADITIVO DIETARIO FUNCIONAL PA...,CIENCIAS AGRICOLAS,...,NO APLICA,12.0,PERSONA JURIDICA,CECILIA OÑATE,MUJER,ACUIOLIVE SPA,RM,13. METROPOLITANA,90000,NaN
37168,FOVI210064,37154,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,FORTALECIMIENTO DE LAS CAPACIDADES DE FORMACIO...,CIENCIAS NATURALES,...,NO APLICA,12.0,PERSONA JURIDICA,FRANCISCO CERECEDA,HOMBRE,UNIVERSIDAD TECNICA FEDERICO SANTA MARIA,CENTRO,05. VALPARAISO,20000,NaN
37169,FOVI210065,37155,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,RED PARA EL ESTUDIO DEL METABOLISMO ENERGETICO...,CIENCIAS NATURALES,...,NO APLICA,12.0,PERSONA JURIDICA,MAITE CASTRO,MUJER,UNIVERSIDAD AUSTRAL DE CHILE,SUR,14. LOS RIOS,20000,NaN


In [ ]:
df_missing2["NOMBRE_PROYECTO"].str.len()

37056     76
37057    186
37168    181
37169    139
Name: NOMBRE_PROYECTO, dtype: int64

Ahora hay sólo una iteración con problemas.

In [ ]:
n_projects_prompt = 1

for iii in tqdm(range(0, len(df_missing2), n_projects_prompt)):
  current_df = df_missing2.iloc[iii: iii + n_projects_prompt]
  projects = "\n".join(("- " + current_df["NOMBRE_PROYECTO"]).values)
  messages = [
    system_message_prompt, HumanMessage(content=projects)
  ]
  ai_response = chat(messages)
  responses = ai_response.content.split("\n")
  if len(current_df) != len(responses):
    print(f"Distinto número de respuestas que de proyectos")
    print(iii, iii + 10)
  else:
    rows = list(zip(current_df["CODIGO_PROYECTO"].values, current_df["NOMBRE_PROYECTO"], responses))
    with open(filepath, "a") as fp:
      csv_writer = writer(fp)
      csv_writer.writerows(rows)

100%|██████████| 4/4 [00:04<00:00,  1.06s/it]


## Revisar clasificaciones

In [ ]:
filepath = os.path.join(save_dir, "clasificaciones_anid_salud_otra.csv")

In [ ]:
df_class = pd.read_csv(filepath)
df_class

,código,glosa,clasificación
0,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro
1,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro
2,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro
3,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro
4,ED230064,PODCAST ANTARTICA,otro
...,...,...,...
21061,FOVI210067,FORTALECIENDO LA PARTICIPACION CIUDADANA EN EL...,otro
21062,SUC210102,PNX3: UNA REVOLUCION EN EL TRATAMIENTO DEL DOL...,otro.
21063,SUC210108,VALIDACION DE UN ADITIVO DIETARIO FUNCIONAL PA...,otro.
21064,FOVI210064,FORTALECIMIENTO DE LAS CAPACIDADES DE FORMACIO...,otro.


In [ ]:
df_class["clasificación"].value_counts()

otro       15450
salud       5148
- otro       297
- salud      167
otro.          4
Name: clasificación, dtype: int64

In [ ]:
df_class["clasificación_normalizada"] = df_class["clasificación"].str.replace(".", "").str.replace("-", "").str.strip()
df_class["clasificación_normalizada"].value_counts()

<ipython-input-12-b6ce80997fd7>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_class["clasificación_normalizada"] = df_class["clasificación"].str.replace(".", "").str.replace("-", "").str.strip()


otro     15751
salud     5315
Name: clasificación_normalizada, dtype: int64

### Revisar duplicados

In [ ]:
df_class[df_class.duplicated()]

,código,glosa,clasificación,clasificación_normalizada
2825,P112RT0159,PRODUCCION ARTESANAL DE HORTALIZAS DE IV Y V G...,salud,salud
4031,412RT0444,RECUPERACION SOSTENIBLE DEL ESPACIO MINERO,otro,otro
4033,712RT0449,METODOS AVANZADOS PARA LA VISUALIZACION DEL SU...,otro,otro
6941,VIU15P0004,LADRILLOS CERAMICOS TERMICAMENTE MEJORADOS CON...,otro,otro
6944,VIU15P0008,ENSAMBLE: SISTEMA CONSTRUCTIVO DE MONTAJE RAPI...,otro,otro
6945,VIU15P0010,"ALTAMIRA, MURO DE GAVIONES POSTENSADOS",otro,otro
6946,VIU15P0011,EMPAQUETAMIENTO TECNOLOGICO DE SUBMARINO AUTON...,otro,otro
6947,VIU15P0019,DESARROLLO DE TIRAS REACTIVAS PARA EL DIAGNOST...,salud,salud
6948,VIU15P0020,BIOPACKING A PARTIR DE DESECHOS DE LA INDUSTRI...,otro,otro
6949,VIU15P0022,QUEMADOR INDUSTRIAL UTILIZANDO LA INNOVADORA Y...,otro,otro


Notamos que hay varios códigos "SIN INFORMACIÓN" que no habíamos notado. Revisemos eso primero.

In [ ]:
df_valid["CODIGO_PROYECTO"].value_counts()

SIN INFORMACION    32
VIU15P0024          2
VIU15P0139          2
VIU17P0223          2
D11R1002            2
                   ..
SA16I0136           1
SA16I0134           1
SA16I0122           1
SA16I0100           1
CL-2023B-018        1
Name: CODIGO_PROYECTO, Length: 20951, dtype: int64

Fijémonos sólo en los que son desde 2013 en adelante.

In [ ]:
df_valid.loc[(df_valid["CODIGO_PROYECTO"] == "SIN INFORMACION") &
             (df_valid["AGNO_FALLO"] >= 2013), ["NOMBRE_PROYECTO", "AGNO_FALLO"]]

,NOMBRE_PROYECTO,AGNO_FALLO
23760,K-12 AND HIGHER EDUCATION IN CHILE: SCHOOL SIZ...,2013
23761,BUILDING A NATURAL AND SOCIAL SCIENCE COLLABOR...,2013
23762,COLLABORATING TO QUANTIFY THE HEALTH BENEFITS ...,2013
23763,WHEN DOES RETIREMENT OPTIMIZE HEALTH? CAUSAL E...,2013
23764,PARENTAL INTERVENTIONS FOR HOME COMPUTERS IN C...,2013
23765,PUBLIC PROCUREMENT MECHANISMS: FROM PRACTICE T...,2013
23766,LEARNING FROM 27F: A COMPARATIVE ASSESSMENT OF...,2013
23767,DEGRADATION OF TITIN AT SINGLE MOLECULAR LEVEL,2013
23768,LANGUAGE FOR LEARNING: DESIGNING AND PILOT TES...,2013
23769,"PUBERTAL EVENTS, SUBSTANCE USE AND SEXUAL RISK...",2013


Son varios. Por ahora no nos preocuparemos, pero sí nos fijaremos que para analizar duplicados necesitamos tanto el código como la glosa del proyecto.

In [ ]:
df_class2 = df_class[~df_class.duplicated(subset=["código", "glosa"])]
df_class2

,código,glosa,clasificación,clasificación_normalizada
0,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro
1,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro
2,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro
3,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro
4,ED230064,PODCAST ANTARTICA,otro,otro
...,...,...,...,...
21061,FOVI210067,FORTALECIENDO LA PARTICIPACION CIUDADANA EN EL...,otro,otro
21062,SUC210102,PNX3: UNA REVOLUCION EN EL TRATAMIENTO DEL DOL...,otro.,otro
21063,SUC210108,VALIDACION DE UN ADITIVO DIETARIO FUNCIONAL PA...,otro.,otro
21064,FOVI210064,FORTALECIMIENTO DE LAS CAPACIDADES DE FORMACIO...,otro.,otro


In [ ]:
df_class2["id"] = df_class2["código"] + " " + df_class2["glosa"]
df_class2

<ipython-input-22-39f1f2971a42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_class2["id"] = df_class2["código"] + " " + df_class2["glosa"]


,código,glosa,clasificación,clasificación_normalizada,id
0,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro,ECOS190041 ARQUEOLOGIA Y AMBIENTE: INTERACCION...
1,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro,FP200006 IMPLEMENTACION DE FLUJO CONTINUO DE L...
2,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro,FOVI210059 LIQUENES SUBANTARTICOS UN NUEVO CEN...
3,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro,FOVI210061 CONSOLIDACION DE LA RED BINACIONAL ...
4,ED230064,PODCAST ANTARTICA,otro,otro,ED230064 PODCAST ANTARTICA
...,...,...,...,...,...
21061,FOVI210067,FORTALECIENDO LA PARTICIPACION CIUDADANA EN EL...,otro,otro,FOVI210067 FORTALECIENDO LA PARTICIPACION CIUD...
21062,SUC210102,PNX3: UNA REVOLUCION EN EL TRATAMIENTO DEL DOL...,otro.,otro,SUC210102 PNX3: UNA REVOLUCION EN EL TRATAMIEN...
21063,SUC210108,VALIDACION DE UN ADITIVO DIETARIO FUNCIONAL PA...,otro.,otro,SUC210108 VALIDACION DE UN ADITIVO DIETARIO FU...
21064,FOVI210064,FORTALECIMIENTO DE LAS CAPACIDADES DE FORMACIO...,otro.,otro,FOVI210064 FORTALECIMIENTO DE LAS CAPACIDADES ...


In [ ]:
df_valid["id"] = df_valid["CODIGO_PROYECTO"].astype(str) + " " + df_valid["NOMBRE_PROYECTO"]

Parece que hay problemas con los códigos en la db original. Eso lo analizaremos en la próxima parte, pero por ahora, sabemos que sí procesamos cada llave código-nombre. Procedemos a guardar las clasificaciones.

In [ ]:
df_class2 = df_class2[["código", "glosa", "clasificación", "clasificación_normalizada"]]
df_class2

,código,glosa,clasificación,clasificación_normalizada
0,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro
1,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro
2,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro
3,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro
4,ED230064,PODCAST ANTARTICA,otro,otro
...,...,...,...,...
21061,FOVI210067,FORTALECIENDO LA PARTICIPACION CIUDADANA EN EL...,otro,otro
21062,SUC210102,PNX3: UNA REVOLUCION EN EL TRATAMIENTO DEL DOL...,otro.,otro
21063,SUC210108,VALIDACION DE UN ADITIVO DIETARIO FUNCIONAL PA...,otro.,otro
21064,FOVI210064,FORTALECIMIENTO DE LAS CAPACIDADES DE FORMACIO...,otro.,otro


In [ ]:
df_class2.to_csv(os.path.join(data_dir, "clasificaciones_anid_salud_otro_limpia.csv"), index=False)